In [1]:
import src.inference.vllm_inference as inference
import src.prompt.mbpp as mbpp
import src.prompt.humaneval as humaneval
import src.benchmarks as benchmarks
import src.metrics as metrics
from pprint import pprint

In [2]:
llm, tokenizer, sampling_params = inference.setup_model(use_sft=True)

INFO 01-17 14:02:23 [utils.py:253] non-default args: {'trust_remote_code': True, 'seed': 42, 'max_model_len': 2048, 'enable_prefix_caching': False, 'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'enable_lora': True, 'max_lora_rank': 64, 'model': './models/qwen3-0.6b'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 01-17 14:02:23 [model.py:514] Resolved architecture: Qwen3ForCausalLM
INFO 01-17 14:02:23 [model.py:1661] Using max model len 2048
INFO 01-17 14:02:27 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-17 14:02:27 [interface.py:465] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:27 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='./models/qwen3-0.6b', speculative_config=None, tokenizer='./models/qwen3-0.6b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', d

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=30208) INFO 01-17 14:02:31 [default_loader.py:308] Loading weights took 1.68 seconds
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:31 [punica_selector.py:20] Using PunicaWrapperGPU.
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:32 [gpu_model_runner.py:3659] Model loading took 1.1957 GiB memory and 2.184672 seconds
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:42 [backends.py:643] Using cache directory: /root/.cache/vllm/torch_compile_cache/9666e89cb7/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:42 [backends.py:703] Dynamo bytecode transform time: 9.65 s
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:45 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:47 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 2.09 s
(EngineCore_DP0 pid=30208) INFO 01-17 14:02:47 [monitor.py:34] torch.compile takes 11.74 s in total
(EngineCore_DP0 pid=30208) INFO 01-17 

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE):   0%|          | 0/102 [00:00<?, ?it/s]

(EngineCore_DP0 pid=30208) WARNING 01-17 14:02:49 [utils.py:250] Using default LoRA kernel configs


Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 102/102 [00:06<00:00, 15.34it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 70/70 [00:04<00:00, 16.04it/s]


(EngineCore_DP0 pid=30208) INFO 01-17 14:03:00 [gpu_model_runner.py:4587] Graph capturing finished in 12 secs, took 1.17 GiB
(EngineCore_DP0 pid=30208) INFO 01-17 14:03:00 [core.py:259] init engine (profile, create kv cache, warmup model) took 28.50 seconds
INFO 01-17 14:03:01 [llm.py:360] Supported tasks: ['generate']


In [3]:
mbpp_data = mbpp.get_dataset()
humaneval_data = humaneval.get_dataset()

In [4]:
display(mbpp_data)
pprint(mbpp_data['train'][0])

DatasetDict({
    train: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 120
    })
    test: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 257
    })
    validation: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 43
    })
    prompt: Dataset({
        features: ['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'],
        num_rows: 7
    })
})

{'code': 'def first_repeated_char(str1):\n'
         '  for index,c in enumerate(str1):\n'
         '    if str1[:index+1].count(c) > 1:\n'
         '      return c',
 'prompt': 'Write a python function to find the first repeated character in a '
           'given string.',
 'source_file': 'Benchmark Questions Verification V2.ipynb',
 'task_id': 602,
 'test_imports': [],
 'test_list': ['assert first_repeated_char("abcabc") == "a"',
               'assert first_repeated_char("abc") == None',
               'assert first_repeated_char("123123") == "1"']}


In [5]:
display(humaneval_data)
pprint(humaneval_data['test'][0])

DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point'],
        num_rows: 164
    })
})

{'canonical_solution': '    for idx, elem in enumerate(numbers):\n'
                       '        for idx2, elem2 in enumerate(numbers):\n'
                       '            if idx != idx2:\n'
                       '                distance = abs(elem - elem2)\n'
                       '                if distance < threshold:\n'
                       '                    return True\n'
                       '\n'
                       '    return False\n',
 'entry_point': 'has_close_elements',
 'prompt': 'from typing import List\n'
           '\n'
           '\n'
           'def has_close_elements(numbers: List[float], threshold: float) -> '
           'bool:\n'
           '    """ Check if in given list of numbers, are any two numbers '
           'closer to each other than\n'
           '    given threshold.\n'
           '    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n'
           '    False\n'
           '    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n'

In [6]:
mbpp_results, mbpp_metrics = benchmarks.run_mbpp_bench(
    mbpp_data, "test", llm, tokenizer, sampling_params
)

Starting MBPP benchmark on 257 samples...


  0%|          | 0/1 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/257 [00:00<?, ?it/s]

WARNING 01-17 14:03:07 [input_processor.py:250] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.


Processed prompts:   0%|          | 0/257 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [7]:
mbpp_stats = metrics.aggregate_metrics(mbpp_metrics)
print("MBPP Stats:")
pprint(mbpp_stats)

MBPP Stats:
{'mean_%_passed': 0.3414396887159533,
 'mean_pass@1': 0.29961089494163423,
 'total_samples': 257}


In [8]:
print("\nSample Output:")
print(mbpp_results[0].outputs[0].text)


Sample Output:
<think>

</think>

```python
def remove_Occ(s,ch):
    s = s[ch+1:-1]
    return (s)
```


In [9]:
he_results, he_metrics = benchmarks.run_humaneval_bench(
    humaneval_data, "test", llm, tokenizer, sampling_params
)

Starting HumanEval benchmark on 164 samples...


  0%|          | 0/6 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [10]:
he_stats = metrics.aggregate_metrics(he_metrics)
print("HumanEval Stats:")
pprint(he_stats)

HumanEval Stats:
{'mean_%_passed': 0.4171877868066892,
 'mean_pass@1': 0.15853658536585366,
 'total_samples': 164}


In [11]:
mean_entropy = sum(x["entropy"] for x in mbpp_metrics) / len(mbpp_metrics)
print(f"\nMean Entropy (MBPP): {mean_entropy:.4f}")


Mean Entropy (MBPP): 0.0843
